In [2]:
import pymongo
from pymongo import MongoClient
import pprint
import pandas as pd
import json as json
import numpy as np
import findspark
from datetime import datetime
findspark.init('/home/kaiwalya/Desktop/spark-3.1.1-bin-hadoop2.7')
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import *
spark = SparkSession.\
builder.\
appName("pyspark-notebook2").\
config("spark.executor.memory", "1g").\
config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.0").\
getOrCreate()

In [3]:
client = MongoClient("mongodb://127.0.0.1:27017")

In [4]:
db=client.MED_EX_CHAIN
product=db.PRODUCT
manufacturer=db.MANUFACTURER
distributor=db.DISTRIBUTOR
hospital=db.HOSPITAL
user_roles=db.USER_ROLES
correlation=db.CORRELATION_DATA

In [5]:
manufacturer_csv = pd.read_csv("Schema - Master Data/Manufacturer.csv").dropna(how="all")
manufacturer_csv


MANUFACTURER_ID MANUFACTURER_ORG_TYPE              MANUFACTURER_NAME  \
0           M1001           Electronics            MALLINCKRODT update   
1           M1002                Pharma  Medical Action Industries Inc   
2           M1003                Pharma                  ENERGIZER New   
3           M1004                Pharma      PROCTER & GAMBLE/DURACELL   
4           M1005                Pharma              kaiwalya kulkarni   

  MANUFACTURER_SHORT_NAME LAST_UPDATE_USER SOURCE_IDENTIFIER  \
0                     MLK            USER1                S1   
1                     MAL            USER1                S1   
2                     ENG            USER1                S2   
3                    PGDR            USER1                S2   
4                      KK            USER2             KK123   

  COUNTRY_OF_ORIGIN MANUFACTURER_ADDRESS  
0                US          MAN_ADDR123  
1                US          MAN_ADDR124  
2                US          MAN_ADDR125  
3                US          MAN_ADDR126  
4                US          MAN_ADDR127

In [6]:
distributor_csv = pd.read_csv("Schema - Master Data/Distributor.csv").dropna(how="all")
distributor_csv

DISTRIBUTOR_ID DISTRIBUTOR_ORG_TYPE       DISTRIBUTOR_NAME  \
1           D1001          Electronics           mckesson NYC   
2           D1002            Medicines         Owens & Minor    
3           D1003          Electronics        Cardinal Health   
4           D1004            Medicines         Mckesson India   
5           D1005          Electronics    Owens & Minor India   
6           D1006          Automobiles  Cardinal Health India   
7           D1007          Automobiles            mckesson US   
8           D1008          Automobiles       Owens & Minor US   
9           D1009          Automobiles     Cardinal Health US   
10          D1010          Automobiles            mckesson US   

   DISTRIBUTOR_SHORT_NAME LAST_UPDATE_USER SOURCE_IDENTIFIER  \
1                     MKC            USER1                KK   
2                     O&M            USER1                S1   
3                      CH            USER1                S1   
4                     MKC            USER1                S1   
5                     O&M            USER1                S1   
6                      CH            USER2                S2   
7                     MKC            USER2                S2   
8                     O&M            USER2                S2   
9                      CH            USER2                S2   
10                    MKC            USER2                KK   

   COUNTRY_OF_ORIGIN DISTRIBUTOR_ADDRESS  
1                 US           DST_ADDR1  
2                 US           DST_ADDR2  
3                 US           DST_ADDR3  
4                 US           DST_ADDR4  
5                 US           DST_ADDR5  
6                 US           DST_ADDR6  
7                 US           DST_ADDR7  
8                 US           DST_ADDR8  
9                 US           DST_ADDR9  
10                US          DST_ADDR10

In [7]:
hospital_csv = pd.read_csv("Schema - Master Data/Hospital.csv").dropna(how="all")
hospital_csv

HOSPITAL_ID        HOSPITAL_TYPE                   HOSPITAL_NAME  \
0       H1001              Clinics      BETHESDA MEMORIAL HOSPITAL   
1       H1002  Specialty Hospitals   BOCA RATON COMMUNITY HOSPITAL   
2       H1003   Teaching Hospitals     HEALTH FIRST BUS CTR/HOLMES   
3       H1004              Clinics  INDIAN RIVER MEMORIAL HOSPITAL   
4       H1005  Specialty Hospitals  MARTIN MEMORIAL MEDICAL CENTER   
5       H1006   Teaching Hospitals          JUPITER MEDICAL CENTER   

  HOSPITAL_SHORT_NAME LAST_UPDATE_USER SOURCE_IDENTIFIER COUNTRY_OF_ORIGIN  \
0                 BMH            USER1                S1                US   
1                 BCH            USER1                S1                US   
2                HSBC            USER1                S1                US   
3                IRMH            USER1                S1                US   
4                MMMC            USER1                S1                US   
5                 JMC            USER1                S1                US   

  HOSPITAL_ADDRESS  
0           XYZ123  
1           XYZ124  
2           XYZ125  
3           XYZ126  
4           XYZ127  
5           XYZ128

In [8]:
user_csv = pd.read_csv("Schema - Master Data/User.csv").dropna(how="all")
user_csv

USER_ID FIRST_NAME LAST_NAME COMPNAY_NAME USER_ADDRESS         EMAIL_ID  \
1   U1001    F_Name1   L_Name1     mckesson           US  test1@gmail.com   
2   U1001   Kaiwalya  Kulkarni          P&G           US  test2@gmail.com   

  DESIGNATION COUNTRY_OF_ORIGIN  PHONE_NUMBER     USER_ROLE PASSWORD  
1    director                US   818181818.0  Manufacturer       S1  
2     Manager                US   818181819.0   Distributor       S1

### Insert into mongodb with versions

In [16]:
manufacturer_csv = spark.read.csv("Schema - Master Data/Manufacturer_update.csv", header=True).dropna(how="all")
manufacturer_csv.show()

manufacturer_data= spark.read.format("mongo").option("uri",
                                "mongodb://127.0.0.1:27017/MED_EX_CHAIN.MANUFACTURER").load()

manufacturer_data.createOrReplaceTempView('manufacturer_view')
manufacturer_data.show(5, False)

+---------------+---------------------+--------------------+-----------------------+----------------+-----------------+-----------------+--------------------+
|MANUFACTURER_ID|MANUFACTURER_ORG_TYPE|   MANUFACTURER_NAME|MANUFACTURER_SHORT_NAME|LAST_UPDATE_USER|SOURCE_IDENTIFIER|COUNTRY_OF_ORIGIN|MANUFACTURER_ADDRESS|
+---------------+---------------------+--------------------+-----------------------+----------------+-----------------+-----------------+--------------------+
|          M1001|          Electronics|        MALLINCKRODT|                    MLK|           USER1|               S1|               US|       MAN_ADDR 1001|
|          M1002|               Pharma|Medical Action In...|                    MAL|           USER1|               S1|               US|       MAN_ADDR 1002|
|          M1006|               Pharma|    Limitless Health|                     LH|           USER2|               S2|               US|       MAN_ADDR 1006|
|          M1007|               Pharma|Creativ

In [17]:
for i in manufacturer_csv.toJSON().collect():
    new_record = json.loads(i)
    manufacturer_id = new_record['MANUFACTURER_ID']
    query = "select * from manufacturer_view where MANUFACTURER_ID = '"+manufacturer_id+"' and\
                    VERSION = (select max(VERSION) from manufacturer_view \
                    where MANUFACTURER_ID = '"+manufacturer_id+"')"
    available_data= spark.sql(query)
    print(manufacturer_id, available_data.count())
    
    if available_data.count()==0: # no data available, insert with version 1.0
        insert_data = dict(new_record)
        insert_data['VERSION']=1.0
        insert_data['START_DATE'] = str(datetime.now())
        insert_data['END_DATE'] = str('9999-12-31')
        insert_data['LAST_UPDATE_DT'] = str(datetime.now())
        manufacturer.insert_one(insert_data)
        print('insert data with 1 version.')
        
    elif (new_record['MANUFACTURER_NAME'] != available_data.rdd.collect()[0]['MANUFACTURER_NAME'])|\
          (new_record['MANUFACTURER_ADDRESS'] != available_data.rdd.collect()[0]['MANUFACTURER_ADDRESS'])|\
    (new_record['MANUFACTURER_SHORT_NAME'] != available_data.rdd.collect()[0]['MANUFACTURER_SHORT_NAME']): # add conditions for all the columns which are eligible for the SCD.
        manufacturer.update_many({'MANUFACTURER_ID':manufacturer_id , 'VERSION': int(available_data.collect()[0].asDict()['VERSION'])},
                                         {"$set":{'LAST_UPDATE_DT':str(datetime.now()),
                                            'END_DATE':str(datetime.now())}})
        insert_data = dict(new_record)
        insert_data['START_DATE'] = str(datetime.now())
        insert_data['END_DATE'] = str('9999-12-31')
        insert_data['LAST_UPDATE_DT'] = str(datetime.now())
        insert_data['VERSION']=int(available_data.collect()[0].asDict()['VERSION']) + 1
        manufacturer.insert_one(insert_data)
        print('insert data with updated + 1 version.')
                
                 # insert data with new version.
    else:
        update_data = dict(new_record)
        update_data['LAST_UPDATE_DT']= str(datetime.now())
        manufacturer.update_one({'MANUFACTURER_ID':manufacturer_id, 'VERSION': int(available_data.collect()[0].asDict()['VERSION'])},
                                         {"$set": update_data})
        print('update record with same latest version')  # update existing data with same version.

M1001 1
insert data with updated + 1 version.
M1002 1
insert data with updated + 1 version.
M1006 0
insert data with 1 version.
M1007 0
insert data with 1 version.


In [18]:
for i in manufacturer.find():
    pprint.pprint(i)

{'MANUFACTURER_ID': 0,
 'VERSION': 0,
 '_id': ObjectId('607154f0c928b15121562a25')}
{'COUNTRY_OF_ORIGIN': 'US',
 'END_DATE': '2021-04-10 13:13:49.542190',
 'LAST_UPDATE_DT': '2021-04-10 13:13:49.542126',
 'LAST_UPDATE_USER': 'USER1',
 'MANUFACTURER_ADDRESS': 'MAN_ADDR123',
 'MANUFACTURER_ID': 'M1001',
 'MANUFACTURER_NAME': 'MALLINCKRODT',
 'MANUFACTURER_ORG_TYPE': 'Electronics',
 'MANUFACTURER_SHORT_NAME': 'MLK',
 'SOURCE_IDENTIFIER': 'S1',
 'START_DATE': '2021-04-10 13:06:08.515610',
 'VERSION': 1.0,
 '_id': ObjectId('60715568c928b15121562a26')}
{'COUNTRY_OF_ORIGIN': 'US',
 'END_DATE': '2021-04-10 13:13:51.312390',
 'LAST_UPDATE_DT': '2021-04-10 13:13:51.312337',
 'LAST_UPDATE_USER': 'USER1',
 'MANUFACTURER_ADDRESS': 'MAN_ADDR124',
 'MANUFACTURER_ID': 'M1002',
 'MANUFACTURER_NAME': 'Medical Action Industries Inc',
 'MANUFACTURER_ORG_TYPE': 'Pharma',
 'MANUFACTURER_SHORT_NAME': 'MAL',
 'SOURCE_IDENTIFIER': 'S1',
 'START_DATE': '2021-04-10 13:06:09.122036',
 'VERSION': 1.0,
 '_id': Obje

In [204]:
manufacturer_json=json.loads(manufacturer_csv.to_json(orient='records'))
#distributor_json=json.loads(distributor_csv.to_json(orient='records'))
#hospital_json=json.loads(hospital_csv.to_json(orient='records'))
#user_json=json.loads(user_csv.to_json(orient='records'))

In [205]:
manufacturer.insert_many(manufacturer_json)
#distributor.insert_many(distributor_json)
#hospital.insert_many(hospital_json)
#user_roles.insert_many(user_json)

In [10]:
manufacturer.drop()
'''distributor.drop()
hospital.drop()
user_roles.drop()
gpo.drop()
correlation.drop()'''

'distributor.drop()\nhospital.drop()\nuser_roles.drop()\ngpo.drop()\ncorrelation.drop()'

In [11]:
manufacturer.insert_one({'MANUFACTURER_ID': 0, 'VERSION':0})

In [29]:
#manufacturer.drop()
#distributor.drop()
#hospital.drop()
#user_roles.drop()

In [278]:
client = MongoClient("mongodb://127.0.0.1:27017")
db = client.MED_EX_CHAIN
distributor = db.DISTRIBUTOR


file_name = input('Enter File Name: ')
distributor_csv = spark.read.csv("/home/kaiwalya/Desktop/Neo4J/Schema - Master Data/"+file_name, header = True).dropna(how = "all")
distributor_data = spark.read.format("mongo").option("uri",
                                "mongodb://127.0.0.1:27017/MED_EX_CHAIN.DISTRIBUTOR").load()
distributor_data.createOrReplaceTempView('distributor_view')

Enter File Name: j


AnalysisException: Path does not exist: file:/home/kaiwalya/Desktop/Neo4J/Schema - Master Data/j

In [274]:
for i in distributor_csv.toJSON().collect():
    new_record = json.loads(i)
    distributor_id = new_record['DISTRIBUTOR_ID']
    query = "select * from distributor_view \
             where DISTRIBUTOR_ID = '" + distributor_id + "' and\
                   VERSION = (select max(VERSION) from distributor_view \
             where DISTRIBUTOR_ID = '" + distributor_id + "')"
    available_data = spark.sql(query)
    print(distributor_id, available_data.count())
    
    if available_data.count() == 0: # no data available, insert with version 1.0
        insert_data = dict(new_record)
        insert_data['VERSION'] = 1.0
        insert_data['START_DATE'] = str(datetime.now())
        insert_data['END_DATE'] = str('9999-12-31')
        insert_data['LAST_UPDATE_DT'] = str(datetime.now())
        distributor.insert_one(insert_data)
        print('insert data with 1 version.')
        
    elif (new_record['DISTRIBUTOR_NAME'] != available_data.rdd.collect()[0]['DISTRIBUTOR_NAME'])|\
         (new_record['DISTRIBUTOR_ADDRESS'] != available_data.rdd.collect()[0]['DISTRIBUTOR_ADDRESS'])|\
         (new_record['DISTRIBUTOR_SHORT_NAME'] != available_data.rdd.collect()[0]['DISTRIBUTOR_SHORT_NAME']): # add conditions for all the columns which are eligible for the SCD.
        distributor.update_many({'DISTRIBUTOR_ID': distributor_id, 
                                 'VERSION': int(available_data.collect()[0].asDict()['VERSION'])},
                                  {"$set": {'LAST_UPDATE_DT': str(datetime.now()),
                                            'END_DATE': str(datetime.now())}})
        insert_data = dict(new_record)
        insert_data['START_DATE'] = str(datetime.now())
        insert_data['END_DATE'] = str('9999-12-31')
        insert_data['LAST_UPDATE_DT'] = str(datetime.now())
        insert_data['VERSION'] = int(available_data.collect()[0].asDict()['VERSION']) + 1
        distributor.insert_one(insert_data)
        print('insert data with updated + 1 version.')
                
                 # insert data with new version.
    else:
        update_data = dict(new_record)
        update_data['LAST_UPDATE_DT'] = str(datetime.now())
        distributor.update_one({'DISTRIBUTOR_ID': distributor_id, 
                                'VERSION': int(available_data.collect()[0].asDict()['VERSION'])},
                                 {"$set": update_data})
        print('update record with same latest version')  # update existing data with same version.

D1001 1
update record with same latest version
D1002 1
update record with same latest version
D1003 1
update record with same latest version
D1004 1
update record with same latest version
D1005 1
update record with same latest version
D1006 1
update record with same latest version
D1007 1
update record with same latest version
D1008 1
update record with same latest version
D1009 1
update record with same latest version
D1010 1
insert data with updated + 1 version.


In [32]:
distributor.delete_many({})

In [21]:
distributor.insert_one({'DISTRIBUTOR_ID':0 , 'VERSION' : 0})

In [19]:
for i in distributor.find():
    pprint.pprint(i)

{'DISTRIBUTOR_ID': 0, 'VERSION': 0, '_id': ObjectId('606eed33a848054b381b9017')}
{'COUNTRY_OF_ORIGIN': 'US',
 'DISTRIBUTOR_ADDRESS': 'DST_ADDR1',
 'DISTRIBUTOR_ID': 'D1001',
 'DISTRIBUTOR_NAME': 'mckesson',
 'DISTRIBUTOR_ORG_TYPE': 'Electronics',
 'DISTRIBUTOR_SHORT_NAME': 'MKC',
 'END_DATE': '2021-04-08 17:37:13.542310',
 'LAST_UPDATE_DT': '2021-04-08 17:37:13.542233',
 'LAST_UPDATE_USER': 'USER1',
 'SOURCE_IDENTIFIER': 'KK',
 'START_DATE': '2021-04-08 17:27:53.600015',
 'VERSION': 1.0,
 '_id': ObjectId('606eefc125d63b474967ccd6')}
{'COUNTRY_OF_ORIGIN': 'US',
 'DISTRIBUTOR_ADDRESS': 'DST_ADDR2',
 'DISTRIBUTOR_ID': 'D1002',
 'DISTRIBUTOR_NAME': 'Owens & Minor ',
 'DISTRIBUTOR_ORG_TYPE': 'Medicines',
 'DISTRIBUTOR_SHORT_NAME': 'O&M',
 'END_DATE': '2021-04-08 17:37:15.159248',
 'LAST_UPDATE_DT': '2021-04-08 17:37:15.159198',
 'LAST_UPDATE_USER': 'USER1',
 'SOURCE_IDENTIFIER': 'S1',
 'START_DATE': '2021-04-08 17:27:53.977231',
 'VERSION': 1.0,
 '_id': ObjectId('606eefc125d63b474967ccd7')}

In [299]:
client = MongoClient("mongodb://127.0.0.1:27017")
db = client.MED_EX_CHAIN
hospital = db.HOSPITAL


file_name = input('Enter File Name: ')
hospital_csv = spark.read.csv("/home/kaiwalya/Desktop/Neo4J/Schema - Master Data/"+file_name, header = True).dropna(how = "all")
hospital_data = spark.read.format("mongo").option("uri",
                                "mongodb://127.0.0.1:27017/MED_EX_CHAIN.HOSPITAL").load()
hospital_data.createOrReplaceTempView('hospital_view')


Enter File Name: Hospital.csv


In [300]:
for i in hospital_csv.toJSON().collect():
    new_record = json.loads(i)
    hospital_id = new_record['HOSPITAL_ID']
    query = "select * from hospital_view \
             where HOSPITAL_ID = '" + hospital_id + "' and\
                   VERSION = (select max(VERSION) from hospital_view \
             where HOSPITAL_ID = '" + hospital_id + "')"
    available_data = spark.sql(query)
    print(hospital_id, available_data.count())
    
    if available_data.count() == 0: # no data available, insert with version 1.0
        insert_data = dict(new_record)
        insert_data['VERSION'] = 1.0
        insert_data['START_DATE'] = str(datetime.now())
        insert_data['END_DATE'] = str('9999-12-31')
        insert_data['LAST_UPDATE_DT'] = str(datetime.now())
        hospital.insert_one(insert_data)
        print('insert data with 1 version.')
        
    elif (new_record['HOSPITAL_NAME'] != available_data.rdd.collect()[0]['HOSPITAL_NAME'])|\
         (new_record['HOSPITAL_ADDRESS'] != available_data.rdd.collect()[0]['HOSPITAL_ADDRESS'])|\
         (new_record['HOSPITAL_SHORT_NAME'] != available_data.rdd.collect()[0]['HOSPITAL_SHORT_NAME']): # add conditions for all the columns which are eligible for the SCD.
        hospital.update_many({'HOSPITAL_ID': hospital_id, 
                                 'VERSION': int(available_data.collect()[0].asDict()['VERSION'])},
                                  {"$set": {'LAST_UPDATE_DT': str(datetime.now()),
                                            'END_DATE': str(datetime.now())}})
        insert_data = dict(new_record)
        insert_data['START_DATE'] = str(datetime.now())
        insert_data['END_DATE'] = str('9999-12-31')
        insert_data['LAST_UPDATE_DT'] = str(datetime.now())
        insert_data['VERSION'] = int(available_data.collect()[0].asDict()['VERSION']) + 1
        hospital.insert_one(insert_data)
        print('insert data with updated + 1 version.')
                
                 # insert data with new version.
    else:
        update_data = dict(new_record)
        update_data['LAST_UPDATE_DT'] = str(datetime.now())
        hospital.update_one({'HOSPITAL_ID': hospital_id, 
                                'VERSION': int(available_data.collect()[0].asDict()['VERSION'])},
                                 {"$set": update_data})
        print('update record with same latest version')  # update existing data with same version.

H1001 1
insert data with updated + 1 version.
H1002 1
update record with same latest version
H1003 1
insert data with updated + 1 version.
H1004 1
update record with same latest version
H1005 1
update record with same latest version
H1006 1
update record with same latest version


In [23]:
hospital.delete_many({})

In [24]:
hospital.insert_one({'HOSPITAL_ID':0, 'VERSION':0})

In [304]:
for i in hospital.find():
    pprint.pprint(i)

{'HOSPITAL_ID': 0, 'VERSION': 0, '_id': ObjectId('606c9cd23a7f3e4f196c5afe')}
{'COUNTRY_OF_ORIGIN': 'US',
 'END_DATE': '2021-04-06 23:12:39.925571',
 'HOSPITAL_ADDRESS': 'XYZ123 NEW',
 'HOSPITAL_ID': 'H1001',
 'HOSPITAL_NAME': 'BETHESDA MEMORIAL HOSPITAL',
 'HOSPITAL_SHORT_NAME': 'BMH',
 'HOSPITAL_TYPE': 'Clinics',
 'LAST_UPDATE_DT': '2021-04-06 23:12:39.925509',
 'LAST_UPDATE_USER': 'USER1',
 'SOURCE_IDENTIFIER': 'S1',
 'START_DATE': '2021-04-06 23:11:20.247197',
 'VERSION': 1.0,
 '_id': ObjectId('606c9d400ebc70aba6266cf3')}
{'COUNTRY_OF_ORIGIN': 'US',
 'END_DATE': '9999-12-31',
 'HOSPITAL_ADDRESS': 'XYZ124',
 'HOSPITAL_ID': 'H1002',
 'HOSPITAL_NAME': 'BOCA RATON COMMUNITY HOSPITAL',
 'HOSPITAL_SHORT_NAME': 'BCH',
 'HOSPITAL_TYPE': 'Specialty Hospitals',
 'LAST_UPDATE_DT': '2021-04-06 23:12:41.382509',
 'LAST_UPDATE_USER': 'USER1',
 'SOURCE_IDENTIFIER': 'S1',
 'START_DATE': '2021-04-06 23:11:20.638804',
 'VERSION': 1.0,
 '_id': ObjectId('606c9d400ebc70aba6266cf4')}
{'COUNTRY_OF_ORIGIN

In [18]:
client = MongoClient("mongodb://127.0.0.1:27017")
db = client.MED_EX_CHAIN
gpo = db.GPO

file_name = input('Enter File Name: ')
gpo_csv = spark.read.csv("/home/kaiwalya/Desktop/Neo4J/Schema - Master Data/"+file_name, header = True).dropna(how = "all")
gpo_data = spark.read.format("mongo").option("uri",
                                "mongodb://127.0.0.1:27017/MED_EX_CHAIN.GPO").load()
gpo_data.createOrReplaceTempView('gpo_view')

Enter File Name: k


AnalysisException: Path does not exist: file:/home/kaiwalya/Desktop/Neo4J/Schema - Master Data/k

In [322]:
for i in gpo_csv.toJSON().collect():
    new_record = json.loads(i)
    gpo_id = new_record['GPO_ID']
    query = "select * from gpo_view \
             where GPO_ID = '" + gpo_id + "' and\
                   VERSION = (select max(VERSION) from gpo_view \
             where GPO_ID = '" + gpo_id + "')"
    available_data = spark.sql(query)
    print(gpo_id, available_data.count())
    
    if available_data.count() == 0: # no data available, insert with version 1.0
        insert_data = dict(new_record)
        insert_data['VERSION'] = 1.0
        insert_data['START_DATE'] = str(datetime.now())
        insert_data['END_DATE'] = str('9999-12-31')
        insert_data['LAST_UPDATE_DT'] = str(datetime.now())
        gpo.insert_one(insert_data)
        print('insert data with 1 version.')
        
    elif (new_record['GPO_NAME'] != available_data.rdd.collect()[0]['GPO_NAME'])|\
         (new_record['GPO_ADDRESS'] != available_data.rdd.collect()[0]['GPO_ADDRESS'])|\
         (new_record['GPO_SHORT_NAME'] != available_data.rdd.collect()[0]['GPO_SHORT_NAME']): # add conditions for all the columns which are eligible for the SCD.
        gpo.update_many({'GPO_ID': gpo_id, 
                              'VERSION': int(available_data.collect()[0].asDict()['VERSION'])},
                              {"$set": {'LAST_UPDATE_DT': str(datetime.now()),
                                            'END_DATE': str(datetime.now())}})
        insert_data = dict(new_record)
        insert_data['START_DATE'] = str(datetime.now())
        insert_data['END_DATE'] = str('9999-12-31')
        insert_data['LAST_UPDATE_DT'] = str(datetime.now())
        insert_data['VERSION'] = int(available_data.collect()[0].asDict()['VERSION']) + 1
        gpo.insert_one(insert_data)
        print('insert data with updated + 1 version.')
                
                 # insert data with new version.
    else:
        update_data = dict(new_record)
        update_data['LAST_UPDATE_DT'] = str(datetime.now())
        gpo.update_one({'GPO_ID': gpo_id, 
                             'VERSION': int(available_data.collect()[0].asDict()['VERSION'])},
                             {"$set": update_data})
        print('update record with same latest version')  # update existing data with same version.

G1001 1
update record with same latest version
G1002 1
update record with same latest version
G1003 1
update record with same latest version
G1004 1
update record with same latest version
G1005 1
update record with same latest version
G1006 1
update record with same latest version
G1007 1
insert data with updated + 1 version.
G1008 1
insert data with updated + 1 version.
G1010 0
insert data with 1 version.


In [323]:
for i in gpo.find():
    pprint.pprint(i)

{'GPO_ID': 0, 'VERSION': 0, '_id': ObjectId('606c9f263a7f3e4f196c5b0b')}
{'COUNTRY_OF_ORIGIN': 'US',
 'END_DATE': '9999-12-31',
 'GPO_ADDRESS': 'GPO_ADDR1',
 'GPO_ID': 'G1001',
 'GPO_NAME': 'GPO_NAME10',
 'GPO_ORG_TYPE': 'Hospital',
 'GPO_SHORT_NAME': 'GPON10',
 'LAST_UPDATE_DT': '2021-04-06 23:20:42.143427',
 'LAST_UPDATE_USER': 'USER',
 'SOURCE_IDENTIFIER': 'S1',
 'START_DATE': '2021-04-06 23:19:46.158065',
 'VERSION': 1.0,
 '_id': ObjectId('606c9f3a3a7f3e4f196c5b0e')}
{'COUNTRY_OF_ORIGIN': 'US',
 'END_DATE': '9999-12-31',
 'GPO_ADDRESS': 'GPO_ADDR2',
 'GPO_ID': 'G1002',
 'GPO_NAME': 'GPO_NAME11',
 'GPO_ORG_TYPE': 'Pharma',
 'GPO_SHORT_NAME': 'GPON11',
 'LAST_UPDATE_DT': '2021-04-06 23:20:42.579811',
 'LAST_UPDATE_USER': 'USER',
 'SOURCE_IDENTIFIER': 'S2',
 'START_DATE': '2021-04-06 23:19:46.225221',
 'VERSION': 1.0,
 '_id': ObjectId('606c9f3a3a7f3e4f196c5b0f')}
{'COUNTRY_OF_ORIGIN': 'US',
 'END_DATE': '9999-12-31',
 'GPO_ADDRESS': 'GPO_ADDR3',
 'GPO_ID': 'G1003',
 'GPO_NAME': 'GPO_N

In [25]:
gpo.delete_many({})

In [26]:
gpo.insert_one({'GPO_ID':0 , 'VERSION': 0})

In [15]:
client = MongoClient("mongodb://127.0.0.1:27017")
db = client.MED_EX_CHAIN
user_roles = db.USER_ROLES


file_name = input('Enter File Name: ')
user_csv = spark.read.csv("/home/kaiwalya/Desktop/Neo4J/Schema - Master Data/"+file_name, header = True).dropna(how = "all")
user_data = spark.read.format("mongo").option("uri",
                            "mongodb://127.0.0.1:27017/MED_EX_CHAIN.USER_ROLES").load()
user_data.createOrReplaceTempView('user_view')


Enter File Name: User.csv


In [16]:
for i in user_csv.toJSON().collect():
    new_record = json.loads(i)
    user_id = new_record['USER_ID']
    query = "select * from user_view \
             where USER_ID = '" + user_id + "' and\
                   VERSION = (select max(VERSION) from user_view \
             where USER_ID = '" + user_id + "')"
    available_data = spark.sql(query)
    print(user_id, available_data.count())
    
    if available_data.count() == 0: # no data available, insert with version 1.0
        insert_data = dict(new_record)
        insert_data['VERSION'] = 1.0
        insert_data['START_DATE'] = str(datetime.now())
        insert_data['END_DATE'] = str('9999-12-31')
        insert_data['LAST_UPDATE_DT'] = str(datetime.now())
        user_roles.insert_one(insert_data)
        print('insert data with 1 version.')
        
    elif (new_record['FIRST_NAME'] != available_data.rdd.collect()[0]['FIRST_NAME'])|\
         (new_record['USER_ADDRESS'] != available_data.rdd.collect()[0]['USER_ADDRESS'])|\
         (new_record['PHONE_NUMBER'] != available_data.rdd.collect()[0]['PHONE_NUMBER']): # add conditions for all the columns which are eligible for the SCD.
        user_roles.update_many({'USER_ID': user_id, 
                                 'VERSION': int(available_data.collect()[0].asDict()['VERSION'])},
                                  {"$set": {'LAST_UPDATE_DT': str(datetime.now()),
                                            'END_DATE': str(datetime.now())}})
        insert_data = dict(new_record)
        insert_data['START_DATE'] = str(datetime.now())
        insert_data['END_DATE'] = str('9999-12-31')
        insert_data['LAST_UPDATE_DT'] = str(datetime.now())
        insert_data['VERSION'] = int(available_data.collect()[0].asDict()['VERSION']) + 1
        user_roles.insert_one(insert_data)
        print('insert data with updated + 1 version.')
                
                 # insert data with new version.
    else:
        update_data = dict(new_record)
        update_data['LAST_UPDATE_DT'] = str(datetime.now())
        user_roles.update_one({'USER_ID': user_id, 
                                'VERSION': int(available_data.collect()[0].asDict()['VERSION'])},
                                 {"$set": update_data})
        print('update record with same latest version')  # update existing data with same version.

U1001 0
insert data with 1 version.
U1001 0
insert data with 1 version.


In [27]:
user_roles.delete_many({})

In [28]:
user_roles.insert_one({'USER_ID': 0 , 'VERSION':0})

In [332]:
for i in user_roles.find():
    pprint.pprint(i)

{'USER_ID': 0, 'VERSION': 0, '_id': ObjectId('606ca07d3a7f3e4f196c5b1c')}
{'COMPNAY_NAME': 'mckesson',
 'COUNTRY_OF_ORIGIN': 'US',
 'DESIGNATION': 'director',
 'EMAIL_ID': 'test1@gmail.com',
 'END_DATE': '2021-04-06 23:26:05.156192',
 'FIRST_NAME': 'F_Name1',
 'LAST_NAME': 'L_Name1',
 'LAST_UPDATE_DT': '2021-04-06 23:26:05.156144',
 'PASSWORD': 'S1',
 'PHONE_NUMBER': '818181818',
 'START_DATE': '2021-04-06 23:25:33.973023',
 'USER_ADDRESS': 'US',
 'USER_ID': 'U1001',
 'USER_ROLE': 'Manufacturer',
 'VERSION': 1.0,
 '_id': ObjectId('606ca0953a7f3e4f196c5b1e')}
{'COMPNAY_NAME': 'P&G',
 'COUNTRY_OF_ORIGIN': 'US',
 'DESIGNATION': 'Manager',
 'EMAIL_ID': 'test2@gmail.com',
 'END_DATE': '2021-04-06 23:26:05.156192',
 'FIRST_NAME': 'F_Name2',
 'LAST_NAME': 'L_Name2',
 'LAST_UPDATE_DT': '2021-04-06 23:26:05.156144',
 'PASSWORD': 'S1',
 'PHONE_NUMBER': '818181819',
 'START_DATE': '2021-04-06 23:25:34.063212',
 'USER_ADDRESS': 'US',
 'USER_ID': 'U1001',
 'USER_ROLE': 'Distributor',
 'VERSION': 1.

In [16]:
db=client.MED_EX_CHAIN
correlation=db.CORRELATION_DATA

distributor_file = input('Enter distributor product catalog file name: ')

distributor_catalog=pd.read_excel("/home/kaiwalya/Desktop/Neo4J/Product Data/"+distributor_file).dropna(how='all').astype(str)


distr_product_catalog=distributor_catalog[['Account Name', 'Supplier Name',
       'Supplier SKU Number', 'Distributor SKU', 'SKU Description',
       'Hospital Item Number','Distributor Name','Contract Price','Contract Number','Unit Price']]
distr_product_catalog=distr_product_catalog.rename(columns=
            {'Account Name':'HOSPITAL_NAME', 'Supplier Name':'MANUFACTURER_NAME',
             'Supplier SKU Number':'PRODUCT_ID',
             'Distributor SKU':'DISTRIBUTOR_SKU', 'SKU Description':'DESCRIPTION',
       'Hospital Item Number':'HOSPITAL_SKU','Distributor Name':'DISTRIBUTOR_NAME','Contract Price':'CONTRACT_PRICE',
       'Contract Number':'CONTRACT_NUMBER','Unit Price':'BASE_PRICE'})

correlation_data = distr_product_catalog[['PRODUCT_ID','DISTRIBUTOR_NAME','DISTRIBUTOR_SKU','HOSPITAL_NAME','HOSPITAL_SKU'\
					,'MANUFACTURER_NAME','DESCRIPTION','CONTRACT_PRICE','CONTRACT_NUMBER','BASE_PRICE']]


correlation.insert_many(json.loads(correlation_data.to_json(orient='records')))

Enter distributor product catalog file name: CardinalHealth _Product_Catalog.xlsx


In [20]:
for i in correlation.find():
    pprint.pprint(i)

{'BASE_PRICE': '22.09',
 'CONTRACT_NUMBER': 'C1001',
 'CONTRACT_PRICE': '$20',
 'DESCRIPTION': 'Comp Medc 1314NETBND11             ',
 'DISTRIBUTOR_NAME': 'Cardinal Health',
 'DISTRIBUTOR_SKU': 'D1001',
 'HOSPITAL_NAME': 'HEALTH FIRST BUS CTR/HOLMES',
 'HOSPITAL_SKU': '4799',
 'MANUFACTURER_NAME': 'Novaplus®',
 'PRODUCT_ID': '1001',
 '_id': ObjectId('606ef0a9843599a99519db04')}
{'BASE_PRICE': '1.5',
 'CONTRACT_NUMBER': 'C1001',
 'CONTRACT_PRICE': '$21',
 'DESCRIPTION': 'Tyvek Heat Seal Pouch 4 X 8  ',
 'DISTRIBUTOR_NAME': 'Cardinal Health',
 'DISTRIBUTOR_SKU': 'D1002',
 'HOSPITAL_NAME': 'HEALTH FIRST BUS CTR/HOLMES',
 'HOSPITAL_SKU': '57655',
 'MANUFACTURER_NAME': 'Novaplus®',
 'PRODUCT_ID': '1002',
 '_id': ObjectId('606ef0a9843599a99519db05')}
{'BASE_PRICE': '0.77',
 'CONTRACT_NUMBER': 'C1001',
 'CONTRACT_PRICE': '$22',
 'DESCRIPTION': 'Tyvek Pouch  7 1/2X13 Hs ',
 'DISTRIBUTOR_NAME': 'Cardinal Health',
 'DISTRIBUTOR_SKU': 'D1003',
 'HOSPITAL_NAME': 'HEALTH FIRST BUS CTR/HOLMES',
 'HO